In [1]:
import os
import re

import django
import fs.path
import numpy as np
from django.core.exceptions import ObjectDoesNotExist
from fs.osfs import OSFS

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "multidex.settings")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

django.setup()

from plotter.models import *
from plotter_utils import modeldict

In [ ]:
for spec in ZSpec.objects.all():
    spec.delete()
for obs in ZObs.objects.all():
    obs.delete()

In [3]:
ZObs.objects.all()

<QuerySet [<ZObs: sol12_zcam03100>, <ZObs: sol16_zcam03101>, <ZObs: sol28_zcam03102>, <ZObs: sol30_zcam03104>, <ZObs: sol30_zcam03103>, <ZObs: sol30_zcam03103>, <ZObs: sol30_zcam03103>, <ZObs: sol33_zcam03106>, <ZObs: sol33_zcam03106>, <ZObs: sol36_zcam03107>, <ZObs: sol36_zcam03107>, <ZObs: sol37_zcam03108>, <ZObs: sol41_zcam03109>]>

In [4]:
input_fs = OSFS("/home/michael/Desktop/mcam_spect_data_conversion/data/zcam")
output_fs = OSFS('.')
output_image_dir = output_fs.getsyspath("static_in_pro/our_static/img")

In [ ]:
metaframe = pd.read_csv(input_fs.getsyspath('Metadata-marslab.csv'))
metaframe.columns = [column.lower() for column in metaframe.columns]
# add NaNs back in so we can programmatically delete them
metaframe.replace('-', np.nan, inplace=True)
# we're turning these to ints when we ingest them,
# but python doesn't like statements like int('3.0'),
# so turn to float as an intermediate step
for column in [
    'sol', 'site', 'drive', 'rover_elevation', 'target_elevation', 'tau',
    'focal_distance', 'incidence_angle', 'emission_angle',
    'phase_angle', 'l_s', 'site', 'drive', 'lat', 'lon', 'odometry',
]:
    metaframe[column] = metaframe[column].astype('float')
metaframe['ltst'] = metaframe['ltst'].astype('datetime64')

In [ ]:
temp_images = [
    image for image in input_fs.listdir('temp_images')
]

In [ ]:
for ix, row in metaframe.iterrows():
    row.dropna(inplace=True)
    if 'sclk' in row.index:
        row['sclk'] = int(float(row['sclk']))
    obs = ZObs(**dict(zip(row.index,row.values)))
    # this is the canonical prefix for image / spectra files 
    # associate observation with images using the convoluted decision tree
    obs.clean()
    obs.save()
pd.DataFrame(map(modeldict, ZObs.objects.all()))

In [ ]:
# for spec in MSpec.objects.all():
#     spec.delete()

In [10]:
[file for file in input_fs.listdir('')]

['Sol036_zcam03107_SantaCruz2_spectral_ROIs_data.csv',
 'Sol028_zcam03102_Iina_spectral_ROIs_data.csv',
 'Metadata-marslab.csv',
 'Sol030_zcam03104_Keyah_spectral_ROIs_data-marslab.csv',
 'Sol037_zcam03108_Hedgehog_spectral_ROIs_data-marslab.csv',
 'Sol036_zcam03107_SantaCruz1_spectral_ROIs_data-marslab.csv',
 'Sol036_zcam03107_SantaCruz1_spectral_ROIs_data.csv',
 'Sol036_zcam03107_SantaCruz3_spectral_ROIs_data.csv',
 'Sol012_zcam03100_Maaz_spectral_ROIs_data.csv',
 'Mastcam-Z Multispectral Data Sets - Target Multispectral Observations.csv',
 'Sol036_zcam03107_SantaCruz3_spectral_ROIs_data-marslab.csv',
 'Sol036_zcam03107_SantaCruz2_spectral_ROIs_data-marslab.csv',
 'Sol030_zcam03104_Keyah_spectral_ROIs_data.csv',
 'Sol012_zcam03100_Maaz_spectral_ROIs_data-marslab.csv',
 'Sol037_zcam03108_Hedgehog_spectral_ROIs_data.csv',
 'Sol033_zcam03106_Ahyeeh_spectral_ROIs_data.csv',
 'temp_images',
 'Sol028_zcam03102_Iina_spectral_ROIs_data-marslab.csv',
 'Sol033_zcam03106_Ahyeeh_spectral_ROIs_da

In [12]:
# have to manually associate spectra for now because of pointing issue
spec_files = [
    file for file in input_fs.listdir('') if (
        file.endswith('data-marslab.csv') and file.startswith('Sol'))
]
for spec_file in spec_files:
    frame = pd.read_csv(input_fs.getsyspath(spec_file))
    frame.columns = [column.lower() for column in frame.columns]
    for _, row in frame.iterrows():
        row = row.replace(['-','',' '], np.nan).dropna()
        # if there are missing filters anywhere in the column, including for other
        # spectra, pandas will read the column
        # as object / string, which will cause confusion when we
        # compute averaged filters
        for filt in ZSpec.filters:
            if filt in row.index:
                row[filt] = float(row[filt])

        row.drop(['sol','seq_id', 'instrument'], inplace=True)
        if row['float'] == 'Y':
            row['float'] = True
        else:
            row['float'] = False
        metadata = dict(row) | {
            'filename': spec_file
        }
        spectrum = ZSpec(**metadata)
        spectrum.clean()
        spectrum.save()
